In [34]:
import pandas as pd
import csv
import string
from string import digits
import numpy as np

In [35]:
import nltk

## Benz Topic Modeling

In [36]:
df=pd.read_csv("Benz.csv",sep=",")
df.head()

,comment,comment_id,post_id,post_time,post_title,time
0,The 2011 will be the next generation 212 chass...,epki6bw,bv20oj,1.559270e+09,NEED HELP!!!! I own this car. 2009 E350 and I ...,2019-05-31 03:53:31
1,You.. put tire letters on an E350..?\n\n&#x200...,eplqfo4,bv20oj,1.559270e+09,NEED HELP!!!! I own this car. 2009 E350 and I ...,2019-05-31 11:16:10
2,Clean Benz 🙏🏾 dam tires tho.... smh,eppg173,bv20oj,1.559270e+09,NEED HELP!!!! I own this car. 2009 E350 and I ...,2019-06-01 08:41:23
3,I have a ‘10 E63 so it’s the same gen (w212) a...,eps5fyh,bv20oj,1.559270e+09,NEED HELP!!!! I own this car. 2009 E350 and I ...,2019-06-02 01:17:20
4,I don’t like air as I want to change to coil o...,epkk1ua,bv20oj,1.559270e+09,NEED HELP!!!! I own this car. 2009 E350 and I ...,2019-05-31 04:08:30


In [37]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [38]:
data = df.comment.values.tolist()

In [39]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'will', 'be', 'the', 'next', 'generation', 'chassis', 'the', 'is', 'bad', 'ass', 'the', 'interior', 'will', 'be', 'completely', 'different', 'and', 'you', 'will', 'be', 'riding', 'on', 'air', 'suspension', 'the', 'price', 'whatever', 'your', 'willing', 'to', 'pay']]


In [40]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [41]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['next generation chassis bad ass interior completely different ride air suspension price willing pay', 'put tire letter different body style pretty fair price mileage']


In [42]:
data_lemmatized

['next generation chassis bad ass interior completely different ride air suspension price willing pay',
 'put tire letter different body style pretty fair price mileage',
 'tire tho',
 'so same as sure used market say good price range depend how well maintain upper mid thing check headbolt main cause engine failure model use update bolt lmk anymore question re great car sure love',
 'air want change coil over problem current matic now adjust ride lower drive hard then good change ride air then think',
 'back remove clean get point trust when say compliment never car weekly think too wtf then just stupid cool',
 'love tire hate tire want reaction why want day wtf next day awesome cycle continue there daily',
 'appreciate very much how check head bolt easy visual',
 'always suspension swap then lose ability adjust comfort sport turn dial wouldn buy nice car put coil over plan track only car daily drive coil over very comfy mention sport tune suspension more coil over',
 'lose awesome',
 

In [43]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'will', 'be', 'the', 'next', 'generation', 'chassis', 'the', 'is', 'bad', 'ass', 'the', 'interior', 'will', 'be', 'completely', 'different', 'and', 'you', 'will', 'be', 'riding', 'on', 'air_suspension', 'the', 'price', 'whatever', 'your', 'willing', 'to', 'pay']


In [44]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [45]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mow/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend([
                   'great',
                   'good',
                   'from', 
                   'subject',
                   're', 
                   'edu', 'lol',
                   'use','take','love','thing','wrong','never','car',
                   'think','go','feel',
                   'well','also','really','nice','bad','damn','amazing','awesome'])

In [47]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Remove Stop Words
data_lemmatized= remove_stopwords(data_lemmatized)


print(data_lemmatized[:1])

[['next', 'generation', 'chassis', 'ass', 'interior', 'completely', 'different', 'ride', 'price', 'willing', 'pay']]


In [48]:
data_lemmatized

[['next',
  'generation',
  'chassis',
  'ass',
  'interior',
  'completely',
  'different',
  'ride',
  'price',
  'willing',
  'pay'],
 ['put',
  'tire',
  'letter',
  'different',
  'body_style',
  'pretty',
  'fair',
  'price',
  'mileage'],
 ['tire', 'tho'],
 ['sure',
  'used',
  'market',
  'say',
  'price_range',
  'depend',
  'maintain',
  'upper',
  'mid',
  'check',
  'headbolt',
  'main',
  'cause',
  'engine',
  'failure',
  'model',
  'update',
  'bolt',
  'lmk',
  'anymore',
  'question',
  'sure'],
 ['air',
  'want',
  'change',
  'coil_over',
  'problem',
  'current',
  'matic',
  'adjust',
  'ride',
  'lower',
  'drive',
  'hard',
  'change',
  'ride',
  'air'],
 ['back',
  'remove',
  'clean',
  'get',
  'point',
  'trust',
  'say',
  'compliment',
  'weekly',
  'wtf',
  'stupid',
  'cool'],
 ['tire',
  'hate',
  'tire',
  'want',
  'reaction',
  'want',
  'day',
  'wtf',
  'next',
  'day',
  'cycle',
  'continue',
  'daily'],
 ['appreciate', 'much', 'check', 'head', 

## Create the Dictionary and Corpus needed for Topic Modeling

In [49]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [50]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('ass', 1),
  ('chassis', 1),
  ('completely', 1),
  ('different', 1),
  ('generation', 1),
  ('interior', 1),
  ('next', 1),
  ('pay', 1),
  ('price', 1),
  ('ride', 1),
  ('willing', 1)],
 [('different', 1),
  ('price', 1),
  ('body_style', 1),
  ('fair', 1),
  ('letter', 1),
  ('mileage', 1),
  ('pretty', 1),
  ('put', 1),
  ('tire', 1)]]

## Building the Topic Model

In [51]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [52]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.163*"rear" + 0.115*"tire" + 0.114*"pretty" + 0.100*"replace" + '
  '0.074*"check" + 0.049*"gear" + 0.040*"easy" + 0.028*"remove" + 0.027*"job" '
  '+ 0.024*"expect"'),
 (1,
  '0.095*"turn" + 0.085*"probably" + 0.081*"bit" + 0.055*"design" + '
  '0.043*"sport" + 0.039*"manual" + 0.036*"fun" + 0.035*"kid" + 0.030*"track" '
  '+ 0.027*"move"'),
 (2,
  '0.145*"model" + 0.123*"engine" + 0.096*"sure" + 0.093*"end" + 0.079*"mean" '
  '+ 0.054*"clean" + 0.051*"wagon" + 0.033*"like" + 0.029*"depend" + '
  '0.026*"market"'),
 (3,
  '0.113*"first" + 0.073*"side" + 0.065*"door" + 0.053*"top" + 0.048*"around" '
  '+ 0.045*"test" + 0.040*"air" + 0.037*"read" + 0.037*"similar" + '
  '0.033*"correct"'),
 (4,
  '0.449*"look" + 0.060*"coupe" + 0.058*"headlight" + 0.046*"system" + '
  '0.046*"black" + 0.045*"leave" + 0.029*"spend" + 0.026*"case" + 0.021*"park" '
  '+ 0.016*"totally"'),
 (5,
  '0.141*"way" + 0.110*"start" + 0.102*"sound" + 0.088*"give" + 0.083*"road" + '
  '0.066*"ever" + 0.041*

## Compute Model Perplexity and Coherence Score

In [53]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -17.762382255093936

Coherence Score:  0.3251214508990289


## Visualize the topics-keywords

In [54]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')
pyLDAvis.save_html(vis, 'lda_20_Benz.html')

/usr/local/var/pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


## Building LDA Mallet Model

In [55]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/Users/mow/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
from gensim.models.ldamodel import LdaModel
def convertldaGenToldaMallet(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha, eta=0,
    )
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [ ]:
# Visualize the topics
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)


In [57]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(7,
  [('sound', 0.056578947368421055),
   ('option', 0.04447368421052632),
   ('exhaust', 0.03236842105263158),
   ('call', 0.031842105263157894),
   ('hear', 0.029473684210526315),
   ('upgrade', 0.017368421052631578),
   ('leave', 0.017105263157894738),
   ('miss', 0.016052631578947367),
   ('system', 0.016052631578947367),
   ('phone', 0.015789473684210527)]),
 (14,
  [('issue', 0.052011095700416086),
   ('replace', 0.048081368469717986),
   ('problem', 0.042071197411003236),
   ('fix', 0.031668978270920016),
   ('break', 0.022422561257512713),
   ('time', 0.0212667591308368),
   ('oil', 0.019648636153490523),
   ('work', 0.018955154877484975),
   ('system', 0.018261673601479428),
   ('part', 0.0159500693481276)]),
 (3,
  [('time', 0.1022489539748954),
   ('long', 0.04210251046025105),
   ('turn', 0.03844142259414226),
   ('give', 0.036872384937238496),
   ('stop', 0.02981171548117155),
   ('put', 0.02641213389121339),
   ('happen', 0.02641213389121339),
   ('win', 0.0258891213389

## the optimal number of topics for LDA

In [58]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[1]
model_topics = optimal_model.show_topics(formatted=True)
pprint(optimal_model.print_topics(num_words=10))

## Finding the dominant topic in each sentence

In [31]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.1614,"make, year, buy, model, mercede, people, price...",The 2011 will be the next generation 212 chass...
1,1,4.0,0.1537,"wheel, black, front, tire, light, put, hard, s...",You.. put tire letters on an E350..?\n\n&#x200...
2,2,4.0,0.1394,"wheel, black, front, tire, light, put, hard, s...",Clean Benz 🙏🏾 dam tires tho.... smh
3,3,0.0,0.1613,"make, year, buy, model, mercede, people, price...",I have a ‘10 E63 so it’s the same gen (w212) a...
4,4,1.0,0.1810,"part, replace, cheap, change, low, system, fix...",I don’t like air as I want to change to coil o...
5,5,3.0,0.1468,"find, issue, check, option, back, door, side, ...",The back is badge removed so it’s all about cl...
6,6,4.0,0.1516,"wheel, black, front, tire, light, put, hard, s...",Haha. Love the tires or hate the tires; it’s w...
7,7,3.0,0.1763,"find, issue, check, option, back, door, side, ...",I appreciate that very much. How do you check...
8,8,1.0,0.1800,"part, replace, cheap, change, low, system, fix...",You can always do a suspension swap. But then ...
9,9,2.0,0.1394,"delete, beautiful, sound, color, post, rim, ma...",You lost me at “I don’t like air”. \n\nAirmati...


## Find the most representative document for each topic

In [32]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.4680,"make, year, buy, model, mercede, people, price...",What you want and what you can afford and need...
1,1.0,0.5170,"part, replace, cheap, change, low, system, fix...",Hi there! I had a 2000 CL500 and the abc pump ...
2,2.0,0.3221,"delete, beautiful, sound, color, post, rim, ma...","^(Hi, I'm a bot for linking direct images of a..."
3,3.0,0.4537,"find, issue, check, option, back, door, side, ...",So i got a w202 where Someone put in a comman...
4,4.0,0.4130,"wheel, black, front, tire, light, put, hard, s...",> I use Sport+ after the car has warmed up bec...


## Topic distribution across documents

In [33]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics[:20]

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,0.0,"make, year, buy, model, mercede, people, price...",2738.0,0.2429
1.0,4.0,"wheel, black, front, tire, light, put, hard, s...",1139.0,0.1010
2.0,4.0,"wheel, black, front, tire, light, put, hard, s...",1972.0,0.1749
3.0,0.0,"make, year, buy, model, mercede, people, price...",1082.0,0.0960
4.0,1.0,"part, replace, cheap, change, low, system, fix...",1200.0,0.1064
5.0,3.0,"find, issue, check, option, back, door, side, ...",1002.0,0.0889
6.0,4.0,"wheel, black, front, tire, light, put, hard, s...",1172.0,0.1040
7.0,3.0,"find, issue, check, option, back, door, side, ...",968.0,0.0859
8.0,1.0,"part, replace, cheap, change, low, system, fix...",NaN,NaN
9.0,2.0,"delete, beautiful, sound, color, post, rim, ma...",NaN,NaN
